In [1]:
##sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", ACCESS_KEY)
# sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", "")
##sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", SECRET_KEY)
#sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", "")

In [2]:
import pandas as pd
from os import listdir
from os.path import join, basename
import struct
import pickle
import json
import os
from scipy import misc
import datetime as dt
import time
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
# %pylab inline
from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from zoo.pipeline.nnframes.nn_image_reader import *
from zoo.feature.common import *
from zoo.feature.image.imagePreprocessing import *
from zoo.pipeline.nnframes.nn_classifier import *
from zoo.pipeline.api.keras.layers import Dense, Input, Flatten
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.net import *
from zoo.common.nncontext import *
import urllib
from os import path

In [4]:

def scala_T(input_T):
    """
    Helper function for building Inception layers. Transforms a list of numbers to a dictionary with ascending keys 
    and 0 appended to the front. Ignores dictionary inputs. 
    
    :param input_T: either list or dict
    :return: dictionary with ascending keys and 0 appended to front {0: 0, 1: realdata_1, 2: realdata_2, ...}
    """    
    if type(input_T) is list:
        # insert 0 into first index spot, such that the real data starts from index 1
        temp = [0]
        temp.extend(input_T)
        return dict(enumerate(temp))
    # if dictionary, return it back
    return input_T

In [5]:
def Inception_Layer_v1(input_size, config, name_prefix=""):
    """
    Builds the inception-v1 submodule, a local network, that is stacked in the entire architecture when building
    the full model.  
    
    :param input_size: dimensions of input coming into the local network
    :param config: ?
    :param name_prefix: string naming the layers of the particular local network
    :return: concat container object with all of the Sequential layers' ouput concatenated depthwise
    """        
    
    '''
    Concat is a container who concatenates the output of it's submodules along the provided dimension: all submodules 
    take the same inputs, and their output is concatenated.
    '''
    concat = Concat(2)
    
    """
    In the above code, we first create a container Sequential. Then add the layers into the container one by one. The 
    order of the layers in the model is same with the insertion order. 
    
    """
    conv1 = Sequential()
    
    #Adding layes to the conv1 model we jus created
    
    #SpatialConvolution is a module that applies a 2D convolution over an input image.
    conv1.add(SpatialConvolution(input_size, config[1][1], 1, 1, 1, 1).set_name(name_prefix + "1x1"))
    conv1.add(ReLU(True).set_name(name_prefix + "relu_1x1"))
    concat.add(conv1)
    
    conv3 = Sequential()
    conv3.add(SpatialConvolution(input_size, config[2][1], 1, 1, 1, 1).set_name(name_prefix + "3x3_reduce"))
    conv3.add(ReLU(True).set_name(name_prefix + "relu_3x3_reduce"))
    conv3.add(SpatialConvolution(config[2][1], config[2][2], 3, 3, 1, 1, 1, 1).set_name(name_prefix + "3x3"))
    conv3.add(ReLU(True).set_name(name_prefix + "relu_3x3"))
    concat.add(conv3)
    
    
    conv5 = Sequential()
    conv5.add(SpatialConvolution(input_size,config[3][1], 1, 1, 1, 1).set_name(name_prefix + "5x5_reduce"))
    conv5.add(ReLU(True).set_name(name_prefix + "relu_5x5_reduce"))
    conv5.add(SpatialConvolution(config[3][1], config[3][2], 5, 5, 1, 1, 2, 2).set_name(name_prefix + "5x5"))
    conv5.add(ReLU(True).set_name(name_prefix + "relu_5x5"))
    concat.add(conv5)
    
    
    pool = Sequential()
    pool.add(SpatialMaxPooling(3, 3, 1, 1, 1, 1, to_ceil=True).set_name(name_prefix + "pool"))
    pool.add(SpatialConvolution(input_size, config[4][1], 1, 1, 1, 1).set_name(name_prefix + "pool_proj"))
    pool.add(ReLU(True).set_name(name_prefix + "relu_pool_proj"))
    concat.add(pool).set_name(name_prefix + "output")
    return concat

In [6]:
def Inception_v1(class_num):
    model = Sequential()
    model.add(SpatialConvolution(3, 64, 7, 7, 2, 2, 3, 3, 1, False).set_name("conv1/7x7_s2"))
    model.add(ReLU(True).set_name("conv1/relu_7x7"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool1/3x3_s2"))
    model.add(SpatialCrossMapLRN(5, 0.0001, 0.75).set_name("pool1/norm1"))
    model.add(SpatialConvolution(64, 64, 1, 1, 1, 1).set_name("conv2/3x3_reduce"))
    model.add(ReLU(True).set_name("conv2/relu_3x3_reduce"))
    model.add(SpatialConvolution(64, 192, 3, 3, 1, 1, 1, 1).set_name("conv2/3x3"))
    model.add(ReLU(True).set_name("conv2/relu_3x3"))
    model.add(SpatialCrossMapLRN(5, 0.0001, 0.75).set_name("conv2/norm2"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool2/3x3_s2"))
    model.add(Inception_Layer_v1(192, scala_T([scala_T([64]), scala_T(
         [96, 128]), scala_T([16, 32]), scala_T([32])]), "inception_3a/"))
    model.add(Inception_Layer_v1(256, scala_T([scala_T([128]), scala_T(
         [128, 192]), scala_T([32, 96]), scala_T([64])]), "inception_3b/"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True))
    model.add(Inception_Layer_v1(480, scala_T([scala_T([192]), scala_T(
         [96, 208]), scala_T([16, 48]), scala_T([64])]), "inception_4a/"))
    model.add(Inception_Layer_v1(512, scala_T([scala_T([160]), scala_T(
         [112, 224]), scala_T([24, 64]), scala_T([64])]), "inception_4b/"))
    model.add(Inception_Layer_v1(512, scala_T([scala_T([128]), scala_T(
         [128, 256]), scala_T([24, 64]), scala_T([64])]), "inception_4c/"))
    model.add(Inception_Layer_v1(512, scala_T([scala_T([112]), scala_T(
         [144, 288]), scala_T([32, 64]), scala_T([64])]), "inception_4d/"))
    model.add(Inception_Layer_v1(528, scala_T([scala_T([256]), scala_T(
         [160, 320]), scala_T([32, 128]), scala_T([128])]), "inception_4e/"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True))
    model.add(Inception_Layer_v1(832, scala_T([scala_T([256]), scala_T(
         [160, 320]), scala_T([32, 128]), scala_T([128])]), "inception_5a/"))
    model.add(Inception_Layer_v1(832, scala_T([scala_T([384]), scala_T(
         [192, 384]), scala_T([48, 128]), scala_T([128])]), "inception_5b/"))
    model.add(SpatialAveragePooling(7, 7, 1, 1).set_name("pool5/7x7_s1"))
    model.add(Dropout(0.4).set_name("pool5/drop_7x7_s1"))
    model.add(View([1024], num_input_dims=3))
    model.add(Linear(1024, class_num).set_name("loss3/classifier"))
    model.add(LogSoftMax().set_name("loss3/loss3"))
    model.reset()
    return model

In [7]:
from os import path
DATA_ROOT = "/mnt/nobigdl/vegnonveg-full/"
MODEL_ROOT = "/mnt/nobigdl/models/inceptionv1/"
dbutils.fs.mkdirs(MODEL_ROOT)
checkpoint_path = path.join(MODEL_ROOT, "checkpoints")
image_path = DATA_ROOT + "pictures/"
label_path = DATA_ROOT + "labels/vegnonveg-full_labels.csv"
parquet_path = DATA_ROOT + "vegnonveg-full_parquet/"
#parquet_path = DATA_ROOT + "parquet/"
# dbutils.fs.rm(parquet_path, True)

## Initialize BigDL

Initialize BigDL and re-direct Spark logs

In [9]:
#intialize bigdl
init_engine()
redire_spark_logs()

## Read images to parquet file and load to Spark as Image dataframe

Save data to parquet files and load to Spark. Add label to each image.

In [11]:
display(dbutils.fs.ls("/mnt/nobigdl/vegnonveg-full/labels"))

In [12]:
## Option 1 for image loading: use directory path
# image_path = "/mnt/nobigdl/vegnonveg-full/pictures/*"

In [13]:
## Option 2 for image loading: concatenate file paths
# image_path = "/mnt/nobigdl/datasets/vegnonveg/pictures/44226b66-6fdc-440d-a850-63073bdf9682,/mnt/nobigdl/datasets/vegnonveg/pictures/44226b66-6fdc-440d-a850-63073bdf9682"

In [14]:
## This only runs at the first time to generate parquet files
# image_frame = NNImageReader.readImages(image_path, sc, minParitions=32)

In [15]:
# labels_csv = pd.read_csv("/dbfs/mnt/nobigdl/vegnonveg-full/labels/vegnonveg-full_labels.csv")
# s3_uris = labels_csv["s3_uri"].tolist()

In [16]:
# s3_uri_str = ','.join(s3_uris)
# image_frame = NNImageReader.readImages(s3_uri_str, sc, minPartitions=32)

In [17]:
#save dataframe to parquet files
# image_frame.write.parquet(parquet_path)

In [18]:
# load parquet file into Spark cluster
import time
start = time.time()
image_raw_DF = sqlContext.read.parquet(parquet_path)
# image1 = image_raw_DF.take(1)
# print(image1[0])
end = time.time()
print("Load data time is: " + str(end-start) + " seconds")

In [19]:
# print(image_raw_DF.count())

In [20]:
# create dict from label to numeric label
labels_csv = pd.read_csv("/dbfs"+label_path)
unique_labels = labels_csv["label"].unique().tolist()
label_dict = dict(zip(unique_labels, range(1,len(unique_labels)+1)))
label_dict
len(label_dict)

In [21]:
# image_names = [uri.split("/")[-1] for uri in labels_csv["s3_uri"].tolist()]
# labels = labels_csv["label"].tolist()
# name_dict = dict(zip(image_names, [label_dict[code] for code in labels]))
# name_dict
# len(image_names)

In [22]:
# See https://spark.apache.org/docs/latest/sql-programming-guide.html#broadcast-hint-for-sql-queries
  
# create label dataframe
label_raw_DF = sqlContext.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .option("mode", "DROPMALFORMED")\
    .load(label_path)\
    .na.drop()
# label_raw_DF.show()    
get_label = udf(lambda label: float(label_dict[label]), FloatType())
change_name = udf(lambda uid: uid+".jpg", StringType())
labelDF = label_raw_DF.withColumn("label", get_label("label")).withColumn("image_name", change_name("obs_uid"))
# labelDF = label_raw_DF.withColumn("label", get_label("label"))
labelDF.show(truncate=False)
# print(labelDF.count())

In [23]:
# create image data dataframe
get_name = udf(lambda row: row[0].split("/")[-1], StringType())
# get_label = udf(lambda name: float(name_dict[name]), FloatType())
imageDF = image_raw_DF.withColumn("image_name", get_name("image"))
# imageDF.show(truncate=False)

In [24]:
from pyspark.sql.functions import col, broadcast
# image dataframe join with labels
dataDF = imageDF.join(broadcast(labelDF), "image_name", "inner").select("image", 'image_name', "label")
# dataDF.select("image_name", "label").show(truncate=False)

In [25]:
# print(dataDF.count())

## Do Train/Test Split and preprocessing
Split Train/Test split with some ratio and preprocess images.

In [27]:
data = dataDF.randomSplit([0.8, 0.1, 0.1], seed=10)
train_image = data[0]
val_image = data[1]
test_image = data[2]


In [28]:
IMAGE_SIZE = 224
train_transformer = ChainedPreprocessing(
        [RowToImageFeature(), ImageResize(256, 256), ImageCenterCrop(IMAGE_SIZE, IMAGE_SIZE),
         ImageChannelNormalize(123.0, 117.0, 104.0), ImageMatToTensor(), ImageFeatureToTensor()])



In [29]:
# train_size = train_image.count()
# train_size

## Define Model

In [31]:
# Parameters
learning_rate = 0.06
# parameters for 
batch_size = 1200 #depends on dataset and cluster config (e.g. 16 nodes * 30-cores * 2 images each)
no_epochs = 12 #stop when validation accuracy doesn't improve anymore


In [32]:
# get model
pretrained_model_path = path.join(MODEL_ROOT,"bigdl_inception_v1_imagenet_0_4_0.model")
n_classes = len(label_dict)# label categories
full_model = Net.load_bigdl("dbfs:" + pretrained_model_path)
# create a new model by remove layers after pool5/drop_7x7_s1
model = full_model.new_graph(["pool5/drop_7x7_s1"])

inputNode = Input(name="input", shape=(3, 224, 224))
inception = model.to_keras()(inputNode)
flatten = Flatten()(inception)
logits = Dense(n_classes)(flatten)

lrModel = Model(inputNode, logits)

In [33]:
# train model
classifier = NNClassifier(lrModel, CrossEntropyCriterion(), train_transformer) \
        .setLearningRate(learning_rate)\
        .setBatchSize(batch_size)\
        .setMaxEpoch(no_epochs)\
        .setFeaturesCol("image")\
        .setValidation(EveryEpoch(), val_image, [Top1Accuracy()], batch_size)
start = time.time()
trained_model = classifier.fit(train_image)
end = time.time()
print("Optimization Done.")
print("Training time is: %s seconds" % str(end-start))
# + dt.datetime.now().strftime("%Y%m%d-%H%M%S")

In [34]:
trained_model.save(("/dbfs" + path.join(MODEL_ROOT, "model_finetune")), True)

In [35]:
train_size = 795052
throughput = train_size * no_epochs / (end - start)
print("Average throughput is: %s" % str(throughput))

In [36]:
#predict
predict_model = trained_model.setBatchSize(batch_size)
predictionDF = predict_model.transform(test_image)
predictionDF.cache()

In [37]:
predictionDF.show()


In [38]:
predictionDF.select("image_name", "label", "prediction").coalesce(1).write.format('csv').options(header='true', delimiter = ',').save('/mnt/nobigdl/vegnonveg-full/labels/predicted_labels-20180603.csv')

In [39]:
# num_preds = 1
# preds = predictionDF.select("label", "prediction").take(num_preds)
# for idx in range(num_preds):
# #    true_label = str(map_to_label(map_groundtruth_label(truth[idx].label)))
#     true_label = preds[idx][0]
#     pred_label = preds[idx][1]
#     print(str(idx + 1) +')'+ 'Ground Truth label: '+ str(true_label))
#     print(str(idx + 1) + ')'+ 'Predicted label: '+ str(pred_label))
#     print("correct" if true_label == pred_label else "wrong")

In [40]:
'''
Measure Test Accuracy w/Test Set
'''
evaluator = MulticlassClassificationEvaluator(labelCol="label",
                                              predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictionDF)
# expected error should be less than 10%
print("Accuracy = %g " % accuracy)
predictionDF.unpersist()